    Ontology: Contains all objects
    Subclass: Contains only clases in subclass relations
    exp: Explanations of subclasses in ontologies
    (For each subclass there is a exp)

        For each chosen_explanation in exp:
            List_of_candidate_objects: Objects that are IN chosen_explanation AND that are NOT IN subclass relation
            chosen_candidate = func(List_of_candidate_objects)
            signature=chosen_candidate
            chosen_explanation_ontology_1 = forget(chosen_explanation,signature)
            generate_subclasses(chosen_explanation_ontology_1)
            #Possible for one iteration for each explanation

In [1]:
import os
from IPython.display import clear_output

In [41]:
from shutil import move

In [2]:
from subprocess import Popen, PIPE, CalledProcessError
from shutil import copyfile
from os import listdir
from os.path import isfile, join
import re

In [53]:
def move_file(src,dest):
    copyfile(src,dest)
    os.remove(src)

def get_path( path):
    path = os.path.join(*path)
    return(path)

def get_all_files(mypath, _filter='exp-'):
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return ([f for f in onlyfiles if 'exp-' in f])

def get_name(name, prefix='sc', suffix = '.nt', iteration = 0):
    names= name.split('-')
    if len(names) == 1:
        return(f'{prefix}-{name}-{iteration}{suffix}')
    else:
        i = int(names[-1])
        return(f'{prefix}-{names[1]}-{i}{suffix}')
    
def get_subclasses(path, input_file):
    input_ontology = get_path([path,input_file])
    subclass_file = 'subClasses.nt'
    subclass_file_path = get_path([path,subclass_file])
    cmd = 'java -jar kr_functions.jar ' + 'saveAllSubClasses' + " " + input_ontology
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
        subclass_file_path = get_path([path,subclass_file])
        new_subclass_file = get_name(input_file.strip('.owl'))
        new_subclass_path = get_path([path,new_subclass_file])
        move_file(subclass_file_path,new_subclass_path)
        print(f'Subclass file for {input_file} stored at {new_subclass_path}')
        return(new_subclass_path)
    
def get_explanations(path, input_file, subclass_file):
    input_ontology = get_path([path,input_file])
    input_subclass = get_path([path,subclass_file])
    cmd = 'java -jar kr_functions.jar ' + 'saveAllExplanations' + " " + input_ontology + " " + input_subclass
    exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
    new_full_path = get_path([path,exp_folder])
    os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        exp_files = get_all_files(path,_filter = 'exp-')
        for file in exp_files:
            new_path = get_path([path, exp_folder, file])
            old_path = get_path([path,file])
            move_file(old_path,new_path)
        print(f'Explanation files for {input_file} stored at {new_full_path}')
        return(new_full_path)

In [129]:
def forget_from_exp(exp_path, exp_candidates, func = 'min'):

    candidate_choosing = func_dict[func]
    exp_file = exp_path.split('\\')[-1]
    new_file = get_name(exp_file, prefix = f'{func}_signatures', suffix='')
    new_path = get_path([exp_path.strip(exp_file)[:-1],new_file])


    new_ontology_name = get_name(exp_file, prefix=f'{func}_ont', suffix='')
    new_ontology_path = get_path([exp_path.strip(exp_file)[:-1],new_ontology_name])
    

    
    new_subclass_name = get_name(exp_file, prefix=f'{func}_sc', suffix='')
    new_subclass_path = get_path([exp_path.strip(exp_file)[:-1],new_subclass_name])

    os.mkdir(new_path)
    os.mkdir(new_ontology_path)
    os.mkdir(new_subclass_path)
    for exp,candidates in exp_candidates.items():
        chosen = None
#         chosen = candidate_choosing(candidates)

#         signature = create_signature_exp([chosen],new_path, exp)
#         forgotten = forget_exp(exp_path, exp,signature)
#         new_forgotten = get_path([new_ontology_path,forgotten.split('\\')[-1]])
#         move_file(forgotten,new_forgotten)
#         subclass=get_subclasses(new_ontology_path, new_forgotten.split('\\')[-1])
#         new_subclass = get_path([new_subclass_path, subclass.split('\\')[-1]])
#         move_file(subclass,new_subclass)
#         print(new_subclass,subclass)
#         print(new_forgotten)
#         print(signature)
#         break

        try:
            chosen = candidate_choosing(candidates)

            signature = create_signature_exp([chosen],new_path, exp)
            forgotten = forget_exp(exp_path, exp,signature)
            new_forgotten = get_path([new_ontology_path,forgotten.split('\\')[-1]])
            move_file(forgotten,new_forgotten)
            subclass=get_subclasses(new_ontology_path, new_forgotten.split('\\')[-1])
            new_subclass = get_path([new_subclass_path, subclass.split('\\')[-1]])
            move_file(subclass,new_subclass)
#             print(new_forgotten)
#             print(signature)
#             break
        except:
            print(exp,candidates)
    
    return(new_path, new_ontology_path,new_subclass_path)

        
def create_signature_exp(signature,path,ontology_file=None, name = None):
#     print(ontology_file, signature,path)

    if not name:
        name = get_name('-'+ontology_file.strip('.omn'), prefix = 'signature', suffix='.txt')
    
    signature_path = get_path([path,name])
    with open(signature_path, 'w') as f:
        for item in signature:
            f.write("%s\n" % item)
    return signature_path

def forget_exp(path,input_file, signature_file, method = 3, ext = '.omn'):
    method = str(method)
#     path = path.strip('\\datasets')
    input_ontology = get_path([path,input_file])
    signature = get_path([path,signature_file])
    cmd = 'java -cp lethe-standalone.jar uk.ac.man.cs.lethe.internal.application.ForgettingConsoleApplication --owlFile ' + input_ontology + ' --method ' + method  + ' --signature ' + signature
#     new_full_path = get_path([path,exp_folder])
#     os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        new_ontology = get_name('-'+input_file.strip(ext), iteration=1, prefix='ont', suffix='.owl')
        new_ontology_path = get_path([path,new_ontology])
        result_file = 'result.owl'

        move_file(result_file,new_ontology_path)
#         print(f'New ontology saved at {new_ontology_path}')
        return(new_ontology_path)    

In [126]:
def forget(path,input_file, signature_file, method = 3, ext = '.owl'):
    method = str(method)
#     path = path.strip('\\datasets')
    input_ontology = get_path([path,input_file])
    signature = get_path([path,signature_file])
    cmd = 'java -cp lethe-standalone.jar uk.ac.man.cs.lethe.internal.application.ForgettingConsoleApplication --owlFile ' + input_ontology + ' --method ' + method  + ' --signature ' + signature
#     new_full_path = get_path([path,exp_folder])
#     os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
#         new_ontology = get_name(input_file.strip(ext), iteration=1, prefix='d', suffix='.owl')
#         new_ontology_path = get_path([path,new_ontology])
        result_file = 'result.owl'

#         move_file(result_file,new_ontology_path)
#         print(f'New ontology saved at {new_ontology_path}')
        return(result_file)    

In [6]:
from tqdm import tqdm

In [7]:
def get_min(_dict):
    return min(_dict, key=_dict.get)
def get_max(_dict):
    return max(_dict, key=_dict.get)
def get_random(_dict):
    import random
    return random.choice(list(_dict.keys()))
func_dict = {'min':get_min, 'max':get_max, 'rand':get_random}

In [8]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def get_classes_sc(file):
    candidates = []
    candidate_classes ={}
    non_candidates = []
    non_candidate_classes = []
    all_classes = extract_classes(file)
    return(all_classes)

def get_classes_owl(path):
    with open(path) as f:
        text = f.read()
    _ = text.split(r'Object Properties')
    result = []
    regex = r'".+#.+"'
    
    for i in tqdm(_[-1].split('>')):
        t = re.search(regex,i)
        if t:

            result.append(t.string[t.start()+1:t.end()-1])
    # result = list(re.finditer(regex,text))
    # result = [i.string[i.start():i.end()] for i in result]
    result_count = {}
    for i in tqdm(result):
        result_count[i] = result.count(i)
#     _keys = list(result_count.keys())
    # for i in remove:
    #     for k in _keys:
    #         if i in k.lower():
    #               result_count.pop(k, None)
    return(result_count)
    
    


def extract_classes(file, remove = ['subclassof']):
    with open(file) as f:
        text = f.read()
    result = []
    regex = r'<.+#.+'
    for i in tqdm(text.split('>')):
        t = re.search(regex,i)
        if t:

            result.append(t.string[t.start()+1:t.end()])
    # result = list(re.finditer(regex,text))
    # result = [i.string[i.start():i.end()] for i in result]
    result_count = {}
    for i in tqdm(result):
        result_count[i] = result.count(i)
    _keys = list(result_count.keys())
    for i in remove:
        for k in _keys:
            if i in k.lower():
                  result_count.pop(k, None)
        
    return(result_count)
    

In [131]:
def create_signature(signature,path,ontology_file=None, name=None):
    if not name:
        name = get_name(ontology_file.strip('.owl'), prefix = 'signature', suffix='.txt')
    signature_path = get_path([path,name])
    with open(signature_path, 'w') as f:
        for item in signature:
            f.write("%s\n" % item)
    return signature_path



In [10]:
def get_objects_not_in_sc(sc_path, owl_path, ontology_name):
    sc_classes = get_classes_sc(sc_path)
    objects_owl = get_classes_owl(owl_path)
    candidates = [_object for _object in  list(objects_owl.keys()) if _object not in list(sc_classes.keys())]
    candidate_count= {}
    for candidate in candidates:
        #gets count of everything that is in ontology but not in subclass relationship
        if ontology_name in candidate:
            candidate_count[candidate] = objects_owl[candidate]
    return(candidate_count)

In [11]:
def get_candidates_exp(path, not_sc_objects):
    exp_files = get_all_files(path, _filter = 'exp-')
    files = [get_path([path,i]) for i in exp_files]
    exp_candidates = {}
    for file,exp in zip(files,exp_files):
#         with open(file, 'r') as f:
#             text = f.read()
        classes = extract_classes(file)
        candidate_classes = {}
        for _class, count in classes.items():
            if _class in not_sc_objects.keys():
                candidate_classes[_class] = count
        exp_candidates[exp] = candidate_classes

    clear_output()
    return exp_candidates


In [154]:
def move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology):   
    ontology_name = input_ontology.split('.')[0]
    new_path = get_path([path,f'{func}_{ontology_name}'])
    for folder in [signature_path, ontology_path, sc_path]:
        original_path = folder
        new_folder = folder.split('\\')[-1]
        
        full_path = get_path([new_path, new_folder])
        move(original_path,full_path)
    return(new_path)

def forgetting_loop(min_path, candidate_objects):
    ontology_folder = [get_path([min_path, f]) for f in listdir(min_path) if 'ont' in f][0]
    ontology_paths = [get_path([ontology_folder,f]) for f in listdir(ontology_folder) if isfile(join(ontology_folder, f))]
    signature_folder = [get_path([min_path, f]) for f in listdir(min_path) if 'signatures' in f][0]
    singature_paths = [get_path([signature_folder,f]) for f in listdir(signature_folder) if isfile(join(signature_folder, f))]
    new_ontology_paths = []
    for i,j  in zip(ontology_paths, singature_paths):
        file = i.split('\\')[-1]
        name = i.split('\\')[-1].split('.')[0]
        
        signature_file = j.split('\\')[-1]
#         signature_name = i.split('\\')[-1].split('.')[0]
        dest = get_path([min_path,name])
    #     temp = get_subclasses(ontology_folder,file)s
        try:
            os.mkdir(dest)
        except FileExistsError:
            pass
        dest_file = get_path([dest,'ont-0.owl'])
        dest_signature_file  = get_path([dest, 'signature-0.txt'])
        copyfile(i, dest_file)
        copyfile(j, dest_signature_file)
        new_ontology_paths.append(dest)
        

        
    
    for i in new_ontology_paths:
        stop = False
        iteration = 0
        while not stop:

            name = f'ont-{iteration}.owl'
            path_to_ontology = get_path([i, name])
            _temp =  get_classes_owl(path_to_ontology)
            _candidates = [c for c in _temp.keys() if c in candidate_objects.keys()]
            new_candidates = {}
            for c in _candidates:
                new_candidates[c] = _temp[c]
            print(_candidates)
    #         candidate_objects = get_objects_not_in_sc(subclass, get_path([path,input_ontology]), 'untitled-ontology-55')
            if _candidates:
                iteration+=1
                new_name = f'ont-{iteration}.owl'
                signature = func_dict[func](new_candidates)
                signature_name = 'signature-'+new_name.split('-')[1].strip('.owl')+'.txt'

                signature_path = create_signature(signature = [signature], path=i, name=signature_name)
                _new = forget(i,name,signature_name)

                print(i,signature_path)
                print('--------------')
#                 iteration+=1
#                 new_name = f'ont-{iteration}.owl'
                move(_new, get_path([i, new_name]))
            else:
                stop=True
#         stop=True
#     print(i)

    #     break

In [34]:
path = "F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets"
input_ontology = "HarryPotter_book.owl"

In [12]:

subclass=get_subclasses(path, input_ontology)

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\HarryPotter_book.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\HarryPotter_book.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\subClasses.nt
--------


Subclass file for HarryPotter_book.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-HarryPotter_book-0.nt


In [115]:
candidate_objects = get_objects_not_in_sc(subclass, get_path([path,input_ontology]), 'untitled-ontology-55')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 127/127 [00:00<00:00, 127648.36it/s]


In [116]:
candidate_objects

{'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasFriend': 1,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#knows': 3,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasHelped': 1,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet': 10,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf': 3,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#DrumstrangStud': 2,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Gryffindor': 6,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Slytherin': 5,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Halfblood': 2,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Pureblood': 5,
 'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Wizard': 7,
 'http://www.semanticweb.org/vinu/ont

In [15]:
explanations = get_explanations(path, input_ontology, subclass)

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\HarryPotter_book.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\HarryPotter_book.owl
Format : RDF/XML Syntax
--------


Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-HarryPotter_book-0.nt
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-HarryPotter_book-0.nt
Format : Turtle Syntax
--------


Saving all explanations for the subsumptions located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-HarryPotter_book-0.nt



Computing explanation for: <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Rat> rdfs:subClassOf <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Creature>
-> Explanation #1
	 Axiom 1. SubClassOf(<http://www.s

-> Explanation #10
	 Axiom 1. SubClassOf(<http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Pet> <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Creature>)
-> Explanation #10
	 Axiom 1. EquivalentClasses(<http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#HogStudent> ObjectAllValuesFrom(<http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet> <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Creature>) )
	 Axiom 2. EquivalentClasses(<http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Pet> ObjectSomeValuesFrom(<http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf> <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#HogStudent>) )

Computing explanation for: <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#Rat> rdfs:subClassOf <http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontolo

In [16]:
explanations

'F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\exp-HarryPotter_book-0'

In [17]:
exp_candidates = get_candidates_exp(explanations, candidate_objects)

In [18]:
exp_candidates

{'exp-1.omn': {'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet': 1,
  'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf': 1},
 'exp-10.omn': {'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet': 1,
  'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf': 1},
 'exp-11.omn': {},
 'exp-12.omn': {},
 'exp-13.omn': {},
 'exp-2.omn': {},
 'exp-3.omn': {'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet': 1,
  'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf': 1},
 'exp-4.omn': {},
 'exp-5.omn': {},
 'exp-6.omn': {},
 'exp-7.omn': {'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet': 1,
  'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf': 1},
 'exp-8.omn': {'http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet': 1,
  'http://www.semanticw

In [149]:
func = 'min'
signature_path, ontology_path, sc_path = forget_from_exp(explanations, exp_candidates,func=func)

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-HarryPotter_book-0\exp-1.omn...
Simple: 33.33%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_signatures-HarryPotter_book-0\signature-exp-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@14b560f

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet


Forgetting...
Started at Wed Oct 14 10:51:35 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting has |                    

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-HarryPotter_book-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-HarryPotter_book-0\subClasses.nt
--------


Subclass file for ont-exp-7.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_ont-HarryPotter_book-0\sc-exp-7.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-HarryPotter_book-0\exp-8.omn...
Simple: 33.33%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_signatures-HarryPotter_book-0\signature-exp-8.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgettin

In [150]:
min_path = move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology)

In [155]:
forgetting_loop(min_path, candidate_objects)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10027.02it/s]


['http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf']
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-1\ont-0.owl...
Simple: 33.33%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-1\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@162f559

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf


Forgetting...
Started at Wed Oct 14 10:52:44 PKT 2020



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |================================================| 1/1 (100.00%) Forgetting isPetOf

fa

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-10\ont-0.owl...
Simple: 0.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 3
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-10\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@17fd62

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf


Forgetting...
Started at Wed Oct 14 10:52:45 PKT 2020



...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10031.82it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |================================================| 1/1 (100.00%) Forgetting isPetOf

fa

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-8\ont-0.owl...
Simple: 33.33%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book\ont-exp-8\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@162f559

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf


Forgetting...
Started at Wed Oct 14 10:52:47 PKT 2020



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2973.28it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |================================================| 1/1 (100.00%) Forgetting isPetOf

fa

In [156]:
func = 'max'
signature_path, ontology_path, sc_path = forget_from_exp(explanations, exp_candidates,func=func)
max_path = move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology)
forgetting_loop(max_path, candidate_objects)

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-HarryPotter_book-0\exp-1.omn...
Simple: 33.33%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-HarryPotter_book-0\signature-exp-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@14b560f

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet


Forgetting...
Started at Wed Oct 14 10:54:13 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting has |                    

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-HarryPotter_book-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-HarryPotter_book-0\subClasses.nt
--------


Subclass file for ont-exp-7.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-HarryPotter_book-0\sc-exp-7.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-HarryPotter_book-0\exp-8.omn...
Simple: 33.33%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-HarryPotter_book-0\signature-exp-8.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgettin

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-HarryPotter_book-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-HarryPotter_book-0\subClasses.nt
--------


Subclass file for ont-exp-8.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_ont-HarryPotter_book-0\sc-exp-8.nt
exp-9.omn {}
['http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf']


Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book\ont-exp-1\ont-0.owl...
Simple: 33.33%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book\ont-exp-1\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@162f559

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf


Forgetting...
Started at Wed Oct 14 10:54:20 PKT 2020



...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |================================================| 1/1 (100.00%) Forgetting isPetOf

fa

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book\ont-exp-8\ont-0.owl...
Simple: 33.33%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book\ont-exp-8\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@162f559

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf


Forgetting...
Started at Wed Oct 14 10:54:23 PKT 2020



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |================================================| 1/1 (100.00%) Forgetting isPetOf

fa

In [157]:
func = 'rand'
signature_path, ontology_path, sc_path = forget_from_exp(explanations, exp_candidates,func=func)
rand_path = move_folders(signature_path, ontology_path, sc_path, func, path, input_ontology)
forgetting_loop(rand_path, candidate_objects)

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-HarryPotter_book-0\exp-1.omn...
Simple: 33.33%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_signatures-HarryPotter_book-0\signature-exp-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@14b560f

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#isPetOf


Forgetting...
Started at Wed Oct 14 10:54:24 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting isPet |                

Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-HarryPotter_book-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-HarryPotter_book-0\subClasses.nt
--------


Subclass file for ont-exp-7.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_ont-HarryPotter_book-0\sc-exp-7.nt
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-HarryPotter_book-0\exp-8.omn...
Simple: 33.33%, EL: 66.67%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 6
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_signatures-HarryPotter_book-0\signature-exp-8.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forge

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<?, ?it/s]

exp-9.omn {}
['http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet']


Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_HarryPotter_book\ont-exp-1\ont-0.owl...
Simple: 33.33%, EL: 33.33%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_HarryPotter_book\ont-exp-1\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@162f559

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet


Forgetting...
Started at Wed Oct 14 10:54:32 PKT 2020



...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 11024.45it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |                                                | 0/1 (0.00%) Forgetting isPetOf                                                                                   |================================================| 1/1 (100.00%) Forgetting isPetOf

fa

Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_HarryPotter_book\ont-exp-8\ont-0.owl...
Simple: 33.33%, EL: 33.33%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 3, RBox: 0, ABox: 0 Signature: 5
converting...
Using method 3 (uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_HarryPotter_book\ont-exp-8\signature-1.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.ABoxForgetter$@162f559

Input Number of Axioms: 3
Input Average Axiom size: 4.333333333333333
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.semanticweb.org/vinu/ontologies/2014/2/untitled-ontology-55#hasPet


Forgetting...
Started at Wed Oct 14 10:54:35 PKT 2020



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

 |                                                | 0/1 (0.00%)                                                                  |                                                | 0/1 (0.00%) Forgetting hasPet                                                                                  |                                                | 0/1 (0.00%) Forgetting hasPet                                                                                  |================================================| 1/1 (100.00%) Forgetting hasPet

false


Fin

In [160]:
from difflib import SequenceMatcher

def similar(a, b):
    with open(a, 'r') as f:
        a = f.read()
    with open(b, 'r') as f:
        b = f.read() 
#     print((len(b.split(r'Object Properties'))))
#     print(len(b.splitlines()))
    if (len(a.splitlines())<= 12) or (len(b.splitlines()) <= 12):
        return(-1)
    
    return SequenceMatcher(None, a, b).ratio()

In [161]:
import statistics
def calc_avg_similarity(path, func,exclude = '_ont'):
    ontology_paths = [get_path([path,f ])for f in listdir(path) if 'ont' in f and exclude not in f]
    explanation_names = [f for f in listdir(path) if 'ont' in f and exclude not in f]
    result_dict = {}
    for _path,name in zip(ontology_paths, explanation_names):
        ontologies = [get_path([_path,f]) for f in listdir(_path) if isfile(get_path([_path,f])) and '.owl' in f]
        similarities = []
        print(f'Calculating similarities in {name}')
        if len(ontologies) < 2:
#             print('here')
            result_dict[f'{name}'] = -1
        for i,j in enumerate(range(1,len(ontologies))):
#             print(i,j)
            similarities.append(similar(ontologies[i],ontologies[j]))
        similarities = [i for i in similarities if i != -1]
        if not similarities:
#             print('here')
            result_dict[f'{name}'] = -1
        else:
            result_dict[f'{name}'] = statistics.mean(similarities)
#         print(similarities)
    return(result_dict)

In [162]:
max_dict = calc_avg_similarity(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_HarryPotter_book',
                   func ='max')

Calculating similarities in ont-exp-1
Calculating similarities in ont-exp-10
Calculating similarities in ont-exp-3
Calculating similarities in ont-exp-7
Calculating similarities in ont-exp-8


In [163]:
max_dict

{'ont-exp-1': 0.5821850393700787,
 'ont-exp-10': -1,
 'ont-exp-3': 0.5831692913385826,
 'ont-exp-7': 0.5821850393700787,
 'ont-exp-8': 0.5825958702064897}

In [169]:
min_dict = calc_avg_similarity(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_HarryPotter_book',
                   func ='min')

Calculating similarities in ont-exp-1
Calculating similarities in ont-exp-10
Calculating similarities in ont-exp-3
Calculating similarities in ont-exp-7
Calculating similarities in ont-exp-8


In [170]:
min_dict

{'ont-exp-1': 0.5821850393700787,
 'ont-exp-10': -1,
 'ont-exp-3': 0.5831692913385826,
 'ont-exp-7': 0.5821850393700787,
 'ont-exp-8': 0.5825958702064897}

In [171]:
rand_dict = calc_avg_similarity(r'F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_HarryPotter_book',
                   func ='rand')

Calculating similarities in ont-exp-1
Calculating similarities in ont-exp-10
Calculating similarities in ont-exp-3
Calculating similarities in ont-exp-7
Calculating similarities in ont-exp-8


In [172]:
rand_dict

{'ont-exp-1': 0.5549566440121866,
 'ont-exp-10': -1,
 'ont-exp-3': 0.5831692913385826,
 'ont-exp-7': 0.5821850393700787,
 'ont-exp-8': 0.5553734488410208}

In [173]:
df =  pd.DataFrame([min_dict,max_dict,rand_dict]).T
df.columns = ['min','max','rand']
df.index.rename( 'ontology_generation', inplace=True)

In [174]:
df

,min,max,rand
ontology_generation,,,
ont-exp-1,0.582185,0.582185,0.554957
ont-exp-10,-1.000000,-1.000000,-1.000000
ont-exp-3,0.583169,0.583169,0.583169
ont-exp-7,0.582185,0.582185,0.582185
ont-exp-8,0.582596,0.582596,0.555373


In [175]:
df.to_csv('HarryPotter_book_results.csv')